In [17]:
import pandas as pd
import gensim
import pprint
from gensim import corpora
from gensim.utils import simple_preprocess

In [18]:
og_df = pd.read_pickle("News_ds_cleaned.pkl")

## Making a df with relevant entries

In [19]:
## Conjoin categories which are essentially the same, just different names; specifically
## "PARENTS" and "PARENTING"  |  "WELLNESS" and "HEALTY LIVING"
#og_df.loc[og_df.category == "PARENTS", 'category'] = "PARENTING"
#og_df.loc[og_df.category == "HEALTHY LIVING", 'category'] = "WELLNESS"

In [20]:
## create new df with only categories we want
curated_cats = ["POLITICS", "WELLNESS", "ENTERTAINMENT", "TRAVEL", "PARENTING", "STYLE & BEAUTY",
                "QUEER VOICES", "FOOD & DRINK", "BUSINESS", "SPORTS", "BLACK VOICES", "WORLD NEWS"]

df = og_df.loc[og_df['category'].isin(curated_cats)]

In [21]:
## drop entries where headline/short_description ends up null
df = df[df['short_description_cleaned'].notna()]
df = df[df['headline_cleaned'].notna()]

In [22]:
df.head(2)

,link,headline,category,short_description,authors,date,headline_cleaned,short_description_cleaned
3,https://www.huffpost.com/entry/funniest-parent...,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to...",Caroline Bologna,2022-09-23,funny tweet parent week sept. 17 23,accidentally put grow toothpaste toddler tooth...
7,https://www.huffpost.com/entry/puerto-rico-wat...,Puerto Ricans Desperate For Water After Hurric...,WORLD NEWS,More than half a million people remained witho...,"DÁNICA COTO, AP",2022-09-22,puerto ricans desperate water hurricane fiona ...,half million people remain without water servi...


In [23]:
## reset the indices
## Need this in order to accurately match the rows in this df with their if-tdf vectors
df = df.reset_index(drop=True)

## drop the second column of indices
#df.drop('Unnamed: 0', axis=1, inplace=True)

In [24]:
## create new column with two cleaned cells concatenated
df['cleaned_words'] = df['headline_cleaned'] + ' ' + df['short_description_cleaned']
df['cleaned_words_withsw'] = df['headline_cleaned_withsw'] + ' ' + df['short_description_cleaned_withsw']
df=df[df['cleaned_words']!=' ']

KeyError: 'headline_cleaned_withsw'

In [ ]:
df.head(2)

In [ ]:
df.info()

In [ ]:
 df.groupby("category").count().sort_values(by=["link"])

In [ ]:
## rename single word columns in df 
## (it causes problems if one of the words from text is same as another column name)
df = df.rename(columns={"link":"link_col", 
                        "headline":"headline_col", 
                        "category":"category_col",
                        "authors":"authors_col", 
                        "date":"date_col"})

In [ ]:
## Train_test_split
from sklearn.model_selection import train_test_split

news_train, news_test = train_test_split(df,
                                         random_state = 546,
                                         shuffle = True,
                                         test_size = .2,
                                         stratify = df['category_col'])

In [ ]:
## reset the indices
## Need this in order to accurately match the rows in this df with their if-tdf vectors
news_train = news_train.reset_index(drop=True)
news_test = news_test.reset_index(drop=True)

In [ ]:
news_ttrain, news_tvalid = train_test_split(news_train,
                                         random_state = 305,
                                         shuffle = True,
                                         test_size = .2,
                                         stratify = news_train['category_col'])

In [ ]:
news_ttrain.to_pickle('./news_ttrain.pkl')
news_tvalid.to_pickle('./news_tvalid.pkl')
news_train.to_pickle('./news_train.pkl') 
news_test.to_pickle('./news_test.pkl') 